# $$CatBoost\ Tutorial$$

In this tutorial we would explore some base cases of using catboost, such as model training, cross-validation and predicting, as well as some useful features like early stopping,  snapshot support, feature importances and parameters tuning.

## $$1.\ Data\ Preparation$$
### 1.1 Data Loading
The data for this tutorial can be obtained from [this page](https://www.kaggle.com/c/titanic/data) (you would have to register a kaggle account or just login with facebook or google+)

In [12]:
# ! pip install catboost --user
# ! conda install -c conda-forge catboost 

In [6]:
import catboost
import numpy as np
import pandas as pd

train_df = pd.read_csv('./train.csv')
test_df = pd.read_csv('./test.csv')

train_df.head()

PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   

                                                Name     Sex   Age  SibSp  \
0                            Braund, Mr. Owen Harris    male  22.0      1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                             Heikkinen, Miss. Laina  female  26.0      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                           Allen, Mr. William Henry    male  35.0      0   

   Parch            Ticket     Fare Cabin Embarked  
0      0         A/5 21171   7.2500   NaN        S  
1      0          PC 17599  71.2833   C85        C  
2      0  STON/O2. 3101282   7.9250   NaN        S  
3      0            113803  53.1000  C123        S  
4      0            373450   8.0500   NaN        S

### 1.2 Feature Preparation
First of all let's check how many absent values do we have:

In [7]:
null_value_stats = train_df.isnull().sum(axis=0)
null_value_stats[null_value_stats != 0]

Age         177
Cabin       687
Embarked      2
dtype: int64

As we cat see, **`Age`**, **`Cabin`** and **`Embarked`** indeed have some missing values, so let's fill them with some number way out of their distributions - so the model would be able to easily distinguish between them and take it into account:

In [8]:
train_df.fillna(-999, inplace=True)
test_df.fillna(-999, inplace=True)

Now let's separate features and label variable:

In [9]:
X = train_df.drop('Survived', axis=1)
y = train_df.Survived

Pay attention that our features are of differnt types - some of them are numeric, some are categorical, and some are even just strings, which normally should be handled in some specific way (for example encoded with bag-of-words representation). But in our case we could treat these string features just as categorical one - all the heavy lifting is done inside CatBoost. How cool is that? :)

In [10]:
print(X.dtypes)

categorical_features_indices = np.where(X.dtypes != np.float)[0]

PassengerId      int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object


### 1.3 Data Splitting
Let's split the train data into training and validation sets.

In [11]:
from sklearn.model_selection import train_test_split

X_train, X_validation, y_train, y_validation = train_test_split(X, y, train_size=0.75, random_state=42)

X_test = test_df

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


## $$2.\ CatBoost\ Basics$$
If you have not already installed ***CatBoost***, you can do so by running the following cell (pre-uncomment it). 

Also if you want to draw plots of training you should install **`ipywidgets`** package and run special command before launching jupyter notebook:
```
$ pip install ipywidgets
$ jupyter nbextension enable --py widgetsnbextension
```
We recommend doing this for more convenience.

Let's make necessary imports.

In [13]:
from catboost import CatBoostClassifier, Pool, cv
from sklearn.metrics import accuracy_score

### 2.1 Model Training
Now let's create the model itself: We would go here with default parameters (as they provide a _really_ good baseline almost all the time), the only thing We would like to specify here is `custom_loss` parameter, as this would give us an ability to see what's going on in terms of this competition metric - accuracy, as well as to be able to watch for logloss, as it would be more smooth on dataset of such size.

In [14]:
model = CatBoostClassifier(
    custom_loss=['Accuracy'],
    random_seed=42,
    logging_level='Silent'
)

In [15]:
model.fit(
    X_train, y_train,
    cat_features=categorical_features_indices,
    eval_set=(X_validation, y_validation),
#     logging_level='Verbose',  # you can uncomment this for text output
    plot=True
);

CatboostIpythonWidget(data={'.': {'path': '.', 'name': 'current', 'content': {'passed_iterations': 29, 'total_iterations': 1000, 'rows': {'learn_error': [['iter', 'Logloss', 'Accuracy'], ['0', '0.6761324529', '0.7874251497'], ['1', '0.6601800548', '0.7859281437'], ['2', '0.6470698049', '0.7859281437'], ['3', '0.6348901513', '0.7874251497'], ['4', '0.621820105', '0.8038922156'], ['5', '0.6090063495', '0.8038922156'], ['6', '0.6017223285', '0.8038922156'], ['7', '0.5921549958', '0.8023952096'], ['8', '0.5806300882', '0.8023952096'], ['9', '0.5748666959', '0.8008982036'], ['10', '0.5667782915', '0.7979041916'], ['11', '0.5617686805', '0.7964071856'], ['12', '0.5539055509', '0.8038922156'], ['13', '0.5476711144', '0.8038922156'], ['14', '0.5374872691', '0.8038922156'], ['15', '0.5293872555', '0.8068862275'], ['16', '0.5258735256', '0.8038922156'], ['17', '0.5179795443', '0.8068862275'], ['18', '0.5131818673', '0.8068862275'], ['19', '0.5101009992', '0.8038922156'], ['20', '0.5078122086', '0.8038922156'], ['21', '0.5029539455', '0.8068862275'], ['22', '0.5006659608', '0.8038922156'], ['23', '0.4975885181', '0.8038922156'], ['24', '0.4925802939', '0.8038922156'], ['25', '0.4871629676', '0.8053892216'], ['26', '0.4813552222', '0.8053892216'], ['27', '0.4781139988', '0.8068862275'], ['28', '0.4766365289', '0.8068862275']], 'test_error': [['iter', 'Logloss', 'Accuracy'], ['0', '0.6759033245', '0.7802690583'], ['1', '0.6595133561', '0.7847533632'], ['2', '0.6463191259', '0.7847533632'], ['3', '0.6339869971', '0.7847533632'], ['4', '0.6195920734', '0.8206278027'], ['5', '0.6068750868', '0.8206278027'], ['6', '0.5998099085', '0.8206278027'], ['7', '0.589918541', '0.8206278027'], ['8', '0.5782160324', '0.8206278027'], ['9', '0.5726185739', '0.8206278027'], ['10', '0.5633526582', '0.8161434978'], ['11', '0.5585085804', '0.8116591928'], ['12', '0.5513605005', '0.8206278027'], ['13', '0.5452853309', '0.8206278027'], ['14', '0.5354708253', '0.8206278027'], ['15', '0.5280080588', '0.8206278027'], ['16', '0.5246789933', '0.8206278027'], ['17', '0.5174712093', '0.8206278027'], ['18', '0.5124074664', '0.8206278027'], ['19', '0.5094897975', '0.8206278027'], ['20', '0.5069349474', '0.8206278027'], ['21', '0.5027443571', '0.8206278027'], ['22', '0.500569661', '0.8206278027'], ['23', '0.4975034422', '0.8206278027'], ['24', '0.4933792465', '0.8206278027'], ['25', '0.486859873', '0.8206278027'], ['26', '0.4817730681', '0.8206278027'], ['27', '0.478972481', '0.8206278027'], ['28', '0.4776018059', '0.8206278027']], 'time_left': [['0', '123615', '123'], ['1', '82710', '165'], ['2', '72138', '217'], ['3', '66443', '266'], ['4', '62623', '314'], ['5', '58838', '355'], ['6', '55246', '389'], ['7', '52279', '421'], ['8', '50261', '456'], ['9', '47718', '482'], ['10', '46033', '512'], ['11', '44052', '535'], ['12', '42754', '563'], ['13', '42049', '597'], ['14', '43412', '661'], ['15', '43359', '705'], ['16', '42318', '731'], ['17', '42074', '771'], ['18', '41429', '802'], ['19', '40700', '830'], ['20', '39941', '856'], ['21', '39152', '880'], ['22', '38140', '897'], ['23', '37231', '915'], ['24', '36704', '941'], ['25', '36301', '969'], ['26', '35981', '998'], ['27', '35317', '1017'], ['28', '34635', '1034']], 'meta': {'name': 'experiment', 'iterCount': '1000', 'learnErrorLog': 'learn_error.tsv', 'testErrorLog': 'test_error.tsv', 'timeLeft': 'time_left.tsv', 'loss_Logloss': 'min', 'loss_Accuracy': 'max'}}}}}, layout=Layout(align_self='stretch', height='500px'))

As you can see, it is possible to watch our model learn through verbose output or with nice plots (personally I would definately go with the second option - just check out those plots: you can, for example, zoom in areas of interest!)

With this we can see that the best accuracy value of **0.8385** (on validation set) was acheived on **261** boosting step.

### 2.2 Model Cross-Validation

It is good to validate your model, but to cross-validate it - even better. And also with plots! So with no more words:

In [32]:
pool = Pool(X, y, cat_features=categorical_features_indices)

In [34]:
cv_params = model.get_params()
cv_params.update({
    'loss_function': 'Logloss'
})

cv_data = cv(pool = pool, params= cv_params)

Now we have values of our loss functions at each boosting step averaged by 3 folds, which should provide us with a more accurate estimation of our model performance:

In [52]:
print('Best validation logloss score: {:.2f}±{:.2f} on step {}'.format(
    np.max(cv_data['Logloss_train_avg']),
    cv_data['Logloss_train_stddev'][np.argmax(cv_data['Logloss_train_avg'])],
    np.argmax(cv_data['Logloss_train_avg'])
))

Best validation logloss score: 0.68±0.00 on step 0


In [53]:
print('Precise validation logloss score: {}'.format(np.max(cv_data['Logloss_train_avg'])))

Precise validation logloss score: 0.6765328068824928


As we can see, our initial estimation of performance on single validation fold was too optimistic - that is why cross-validation is so important!

### 2.3 Model Applying
All you have to do to get predictions is

In [54]:
predictions = model.predict(X_test)
predictions_probs = model.predict_proba(X_test)
print(predictions[:10])
print(predictions_probs[:10])

[0. 0. 0. 0. 1. 0. 1. 0. 1. 0.]
[[0.73657739 0.26342261]
 [0.94057787 0.05942213]
 [0.87345814 0.12654186]
 [0.91341905 0.08658095]
 [0.31230328 0.68769672]
 [0.96394822 0.03605178]
 [0.18360192 0.81639808]
 [0.94511558 0.05488442]
 [0.28486361 0.71513639]
 [0.98952229 0.01047771]]


But let's try to get a better predictions and Catboost features help us in it.

## $$3.\ CatBoost\ Features$$
You may have noticed that on model creation step I've specified not only `custom_loss` but also `random_seed` parameter. That was done in order to make this notebook reproducible - by default catboost chooses some random value for seed:

In [55]:
model_without_seed = CatBoostClassifier(iterations=10, logging_level='Silent')
model_without_seed.fit(X, y, cat_features=categorical_features_indices)

print('Random seed assigned for this model: {}'.format(model_without_seed.random_seed_))

Random seed assigned for this model: 77696921960814


Let's define some params and create `Pool` for more convenience. It stores all information about dataset (features, labeles, categorical features indices, weights and and much more).

In [56]:
params = {
    'iterations': 500,
    'learning_rate': 0.1,
    'eval_metric': 'Accuracy',
    'random_seed': 42,
    'logging_level': 'Silent',
    'use_best_model': False
}
train_pool = Pool(X_train, y_train, cat_features=categorical_features_indices)
validate_pool = Pool(X_validation, y_validation, cat_features=categorical_features_indices)

### 3.1 Using the best model
If you essentially have a validation set, it's always better to use the `use_best_model` parameter during training. By default, this parameter is enabled. If it is enabled, the resulting trees ensemble is shrinking to the best iteration.

In [57]:
model = CatBoostClassifier(**params)
model.fit(train_pool, eval_set=validate_pool)

best_model_params = params.copy()
best_model_params.update({
    'use_best_model': True
})
best_model = CatBoostClassifier(**best_model_params)
best_model.fit(train_pool, eval_set=validate_pool);

print('Simple model validation accuracy: {:.4}'.format(
    accuracy_score(y_validation, model.predict(X_validation))
))
print('')

print('Best model validation accuracy: {:.4}'.format(
    accuracy_score(y_validation, best_model.predict(X_validation))
))

Simple model validation accuracy: 0.8206

Best model validation accuracy: 0.8386


### 3.2 Early Stopping
If you essentially have a validation set, it's always easier and better to use early stopping. This feature is similar to the previous one, but only in addition to improving the quality it still saves time.

In [58]:
%%time
model = CatBoostClassifier(**params)
model.fit(train_pool, eval_set=validate_pool)

CPU times: user 23.5 s, sys: 16.7 s, total: 40.2 s
Wall time: 11.6 s


In [59]:
%%time
earlystop_params = params.copy()
earlystop_params.update({
    'od_type': 'Iter',
    'od_wait': 40
})
earlystop_model = CatBoostClassifier(**earlystop_params)
earlystop_model.fit(train_pool, eval_set=validate_pool);

CPU times: user 2.74 s, sys: 2.2 s, total: 4.94 s
Wall time: 1.41 s


In [60]:
print('Simple model tree count: {}'.format(model.tree_count_))
print('Simple model validation accuracy: {:.4}'.format(
    accuracy_score(y_validation, model.predict(X_validation))
))
print('')

print('Early-stopped model tree count: {}'.format(earlystop_model.tree_count_))
print('Early-stopped model validation accuracy: {:.4}'.format(
    accuracy_score(y_validation, earlystop_model.predict(X_validation))
))

Simple model tree count: 500
Simple model validation accuracy: 0.8206

Early-stopped model tree count: 68
Early-stopped model validation accuracy: 0.8161


So we get better quality in a shorter time.

Though as was shown earlier simple validation scheme does not precisely describes model out-of-train score (may be biased because of dataset split) it is still nice to track model improvement dynamics - and thereby as we can see from this example it is really good to stop boosting process earlier (before the overfitting kicks in)

### 3.3 Using Baseline
It is posible to use pre-training results (baseline) for training.

In [61]:
current_params = params.copy()
current_params.update({
    'iterations': 10
})
model = CatBoostClassifier(**current_params).fit(X_train, y_train, categorical_features_indices)
# Get baseline (only with prediction_type='RawFormulaVal')
baseline = model.predict(X_train, prediction_type='RawFormulaVal')
# Fit new model
model.fit(X_train, y_train, categorical_features_indices, baseline=baseline);

### 3.4 Feature Importances
Sometimes it is very important to understand which feature made the greatest contribution to the final result. To do this, the CatBoost model has a `get_feature_importance` method.

In [62]:
model = CatBoostClassifier(iterations=50, random_seed=42, logging_level='Silent').fit(train_pool)
feature_importances = model.get_feature_importance(train_pool)
feature_names = X_train.columns
for score, name in sorted(zip(feature_importances, feature_names), reverse=True):
    print('{}: {}'.format(name, score))

Sex: 57.868947134004834
Pclass: 21.086435286677794
Ticket: 11.915508619190788
Cabin: 5.34828183650701
Fare: 1.2474790068427803
SibSp: 1.0049769225130238
Parch: 0.7893666046173244
Embarked: 0.7390045896464361
PassengerId: 0.0
Name: 0.0
Age: 0.0


This shows that features **`Sex`** and **`Pclass`** had the biggest influence on the result.

### 3.5 Model Saving
It is always really handy to be able to dump your model to disk (especially if training took some time).

In [63]:
model = CatBoostClassifier(iterations=10, random_seed=42, logging_level='Silent').fit(train_pool)
model.save_model('catboost_model.dump')
model = CatBoostClassifier()
model.load_model('catboost_model.dump');

# $$4.\ Parameters\ Tuning$$
While you could always select optimal number of iterations (boosting steps) by cross-validation and learning curve plots, it is also important to play with some of model parameters, and we would like to pay some special attention to `l2_leaf_reg` and `learning_rate`.

In this section, we'll select these parameters using the **`hyperopt`** package.

**conda install -c jaikumarm hyperopt **

In [73]:
import hyperopt

def hyperopt_objective(params):
    model = CatBoostClassifier(
        l2_leaf_reg=int(params['l2_leaf_reg']),
        learning_rate=params['learning_rate'],
        iterations=500,
        eval_metric='Accuracy',
        random_seed=42,
        logging_level='Silent'
    )
    
    cv_data = cv(
        pool = Pool(X, y, cat_features=categorical_features_indices),
        params = model.get_params()
    )
    best_accuracy = np.max(cv_data['Accuracy_train_avg'])
    
    return 1 - best_accuracy # as hyperopt minimises

In [ ]:
from numpy.random import RandomState

params_space = {
    'l2_leaf_reg': hyperopt.hp.qloguniform('l2_leaf_reg', 0, 2, 1),
    'learning_rate': hyperopt.hp.uniform('learning_rate', 1e-3, 5e-1),
}

trials = hyperopt.Trials()

best = hyperopt.fmin(
    hyperopt_objective,
    space=params_space,
    algo=hyperopt.tpe.suggest,
    max_evals=50,
    trials=trials,
    rstate=RandomState(123)
)

print(best)

Now let's get all cv data with best parameters:

In [ ]:
model = CatBoostClassifier(
    l2_leaf_reg=int(best['l2_leaf_reg']),
    learning_rate=best['learning_rate'],
    iterations=500,
    eval_metric='Accuracy',
    random_seed=42,
    logging_level='Silent'
)
cv_data = cv(Pool(X, y, cat_features=categorical_features_indices), model.get_params())

In [ ]:
print('Precise validation accuracy score: {}'.format(np.max(cv_data['test-Accuracy-mean'])))

Recall that with default parameters out cv score was 0.8283, and thereby we have (probably not statistically significant) some improvement.

### Make submission
Now we would re-train our tuned model on all train data that we have

In [ ]:
model.fit(X, y, cat_features=categorical_features_indices)

And finally let's prepare the submission file:

In [ ]:
import pandas as pd
submisstion = pd.DataFrame()
submisstion['PassengerId'] = X_test['PassengerId']
submisstion['Survived'] = model.predict(X_test)

In [ ]:
submisstion.to_csv('submission.csv', index=False)

Finally you can make submission at [Titanic Kaggle competition](https://www.kaggle.com/c/titanic).

That's it! Now you can play around with CatBoost and win some competitions! :)